# Entity Resolution of Player Metrics with Salary Data for NHL Hockey Players#

Before we can do any real work on our ML project, we need to merge our X and Y data which was obtained from different sources. Lets do that now.

In [22]:
import pandas as pd
import os

Step 1 will be to load in our data. Lets get our X's and check the shape. Recall that our X data is NHL performance data, with metrics like Goals, Assists, Games Played, for each player for each year from 2007-08 until 2023-24.

In [23]:
# Import all X data from 2007-08 to 2023-24
X_file_path = '../../Data/Warehouse/PlayerData/parquet' 
player_metrics = pd.read_parquet(X_file_path)

In [24]:
# Drop all data from year 2024
player_metrics = player_metrics[player_metrics['Year'] != 2024].copy()
player_metrics

,Team,Player,Position,GP,TOI/GP,TOI,IPP,Goals,Total Assists,First Assists,...,Birth Country,Nationality,Height (in),Weight (lbs),Draft Year,Draft Team,Draft Round,Round Pick,Overall Draft Position,Year
0,DET,Aaron Downey,R,56,4.5880952380952,256.93333333333,50.00,0,3,3,...,CAN,CAN,73,215,0.0,-,-,-,-,2007
1,NYI,Aaron Johnson,D,30,13.861666666667,415.85,9.52,0,2,0,...,CAN,CAN,73,204,2001.0,CBJ,3,22,85,2007
2,VAN,Aaron Miller,D,57,17.336549707602,988.18333333333,27.27,1,8,1,...,USA,USA,75,210,1989.0,NYR,5,4,88,2007
3,CBJ,Aaron Rome,D,17,18.178431372549,309.03333333333,33.33,1,1,0,...,CAN,CAN,73,220,2002.0,L.A,4,8,104,2007
4,MIN,Aaron Voros,L,55,9.1745454545455,504.6,56.00,7,7,6,...,CAN,CAN,74,210,2001.0,N.J,8,2,229,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15214,VGK,Zach Whitecloud,D,50,17.739,886.95,29.63,2,6,3,...,CAN,CAN,74,207,0.0,-,-,-,-,2023
15215,STL,Zack Bolduc,C,15,10.78,161.7,60.00,1,2,0,...,CAN,CAN,72,187,2021.0,STL,1,17,17,2023
15216,OTT,Zack MacEwen,R,24,6.0611111111111,145.46666666667,50.00,2,1,1,...,CAN,CAN,75,205,0.0,-,-,-,-,2023
15217,OTT,Zack Ostapchuk,C,6,10.408333333333,62.45,0.00,0,0,0,...,CAN,CAN,75,205,2021.0,OTT,2,7,39,2023


In [25]:
player_metrics.columns.tolist()

['Team',
 'Player',
 'Position',
 'GP',
 'TOI/GP',
 'TOI',
 'IPP',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'ixG',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'Goals/GP',
 'Total Assists/GP',
 'First Assists/GP',
 'Second Assists/GP',
 'Total Points/GP',
 'Shots/GP',
 'ixG/GP',
 'iCF/GP',
 'iFF/GP',
 'iSCF/GP',
 'iHDCF/GP',
 'Rush Attempts/GP',
 'Rebounds Created/GP',
 'PIM/GP',
 'Total Penalties/GP',
 'Minor/GP',
 'Major/GP',
 'Misconduct/GP',
 'Penalties Drawn/GP',
 'Giveaways/GP',
 'Takeaways/GP',
 'Hits/GP',
 'Hits Taken/GP',
 'Shots Blocked/GP',
 'Faceoffs Won/GP',
 'Faceoffs Lost/GP',
 'Goals/60',
 'Total Assists/60',
 'First Assists/60',
 'Second Assists/60',
 'Total Points/60',
 'Shots/60',
 'ixG/60',
 'iCF/60',

In [26]:
# Change year column from 2007 to 2007-08 for year 2007 to 2023
player_metrics['Year'] = player_metrics['Year'].apply(lambda x: str(x) + '-' + str(x+1)[2:])

Continuing with step 1, now lets load in our Y values and again check the shape.

In [27]:
# Import all Y data from 2007-08 to 2023-24
Y_file_path = '../../Data/Warehouse/SalaryData/salary_data.csv'

salary_data = pd.read_csv(Y_file_path)

salary_data.shape

(20719, 24)

In [28]:
print(f'Player Metrics Columns: \n {player_metrics.columns}')
print()
print(f'Salary Data Columns: \n {salary_data.columns}')

Player Metrics Columns: 
 Index(['Team', 'Player', 'Position', 'GP', 'TOI/GP', 'TOI', 'IPP', 'Goals',
       'Total Assists', 'First Assists',
       ...
       'Birth Country', 'Nationality', 'Height (in)', 'Weight (lbs)',
       'Draft Year', 'Draft Team', 'Draft Round', 'Round Pick',
       'Overall Draft Position', 'Year'],
      dtype='object', length=211)

Salary Data Columns: 
 Index(['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'COUNTRY', 'POS', 'HANDED',
       'DRAFTED', 'SIGNING TEAM', 'TYPE', 'SIGNING AGE', 'CLAUSE', 'LENGTH',
       'EXPIRY', 'EXP. YEAR', 'CAP HIT', 'AAV', 'SALARY', 'BASE SALARY',
       'S.BONUS', 'P.BONUS', 'season', 'Y_Salary_Cap',
       'Y_Salary_Cap_Percentage'],
      dtype='object')


As we can see, the maximum number of similar rows we can hope to find is 15219 as determined by our X's, since we were able to acquire more Y records than X.

## Assessing Player Metrics ##

Before we merge with our Y's, we should take a look at the player metrics. One thing I am concerned about is the possibility that individual players show up twice in a single year.

In [29]:
# View players that show up twice in one year in player_metrics
player_metrics[player_metrics.duplicated(subset=['Player', 'Year'], keep=False)].sort_values(by=['Player', 'Position', 'Year'])

,Team,Player,Position,GP,TOI/GP,TOI,IPP,Goals,Total Assists,First Assists,...,Birth Country,Nationality,Height (in),Weight (lbs),Draft Year,Draft Team,Draft Round,Round Pick,Overall Draft Position,Year
28,"PHI, T.B",Alexandre Picard,D,24,20.418055555556,490.03333333333,31.58,3,3,0,...,CAN,CAN,75,215,2003.0,PHI,3,17,85,2007-08
881,OTT,Alexandre Picard,D,47,18.869858156028,886.88333333333,28.00,6,8,2,...,CAN,CAN,75,215,2003.0,PHI,3,17,85,2008-09
1766,"CAR, OTT",Alexandre Picard,D,54,18.383950617284,992.73333333333,31.25,4,11,3,...,CAN,CAN,75,215,2003.0,PHI,3,17,85,2009-10
29,CBJ,Alexandre Picard,L,3,6.7666666666667,20.3,0.00,0,0,0,...,CAN,CAN,74,206,2004.0,CBJ,1,8,8,2007-08
882,CBJ,Alexandre Picard,L,15,6.8833333333333,103.25,50.00,0,1,1,...,CAN,CAN,74,206,2004.0,CBJ,1,8,8,2008-09
1765,CBJ,Alexandre Picard,L,9,7.2185185185185,64.966666666667,-,0,0,0,...,CAN,CAN,74,206,2004.0,CBJ,1,8,8,2009-10
12260,CAR,Sebastian Aho,C,56,19.470535714286,1090.35,66.28,24,33,24,...,FIN,None,72,176,2015.0,CAR,2,5,None,2020-21
14202,CAR,Sebastian Aho,C,75,19.495555555556,1462.1666666667,64.42,36,31,24,...,FIN,None,72,176,2015.0,CAR,2,5,None,2022-23
15095,CAR,Sebastian Aho,C,67,19.575373134328,1311.55,72.38,27,49,31,...,FIN,None,72,176,2015.0,CAR,2,5,None,2023-24
13256,CAR,Sebastian Aho,"C, R",79,18.945358649789,1496.6833333333,72.97,37,44,28,...,FIN,None,72,176,2015.0,CAR,2,5,None,2021-22


After reviewing these players and doing some research, I've found that there were actually two Sebastian Aho's and two Alexandre Picard's in the NHL during this time period. So, we can be confident that every player only ever shows up once in the data for each year.

Note that we have already done this process for our salary_data, and chose to drop all records where duplicates existed, since some of them seemed to not make sense.

# Trivial Join #
player_metrics.columns includes Team, Player, Birth City, Birth Country, Nationality, Date of Birth, Year, Round Pick, Position \
salary_data.columns includes PLAYER, TEAM, AGE, POS, Date of Birth

We absolutely must join on Player and Year, but it would be nice to also join on another column in the event that two players with the exact same name are in the league at the same time. Date of Birth is also common to these two tables, and it would be awfully unlikely for two people to be in the league at the same time with the exact same names and birthdates. 

In [30]:
# Merge player_metrics and salary_data on columns Player, Year, Birth Date

# Convert all columns to lower case
player_metrics['Player'] = player_metrics['Player'].str.lower()
player_metrics['Team'] = player_metrics['Team'].str.lower()
player_metrics['Year'] = player_metrics['Year'].str.lower()
player_metrics['Position'] = player_metrics['Position'].str.lower()

salary_data['PLAYER'] = salary_data['PLAYER'].str.lower()
salary_data['TEAM'] = salary_data['TEAM'].str.lower()
salary_data['season'] = salary_data['season'].str.lower()
salary_data['POS'] = salary_data['POS'].str.lower()

We need to check if the data types and structure of both Birth Date columns align.

In [31]:
print(player_metrics["Date of Birth"])
print(salary_data["DATE OF BIRTH"])

0        1974-08-27
1        1983-04-30
2        1971-08-11
3        1983-09-27
4        1981-07-02
            ...    
15214    1996-11-28
15215    2003-02-24
15216    1996-07-08
15217    2003-05-29
15218    1994-01-05
Name: Date of Birth, Length: 15219, dtype: object
0        Feb. 15, 1972
1          May 2, 1980
2        Apr. 28, 1970
3        Mar. 18, 1977
4        Dec. 23, 1979
             ...      
20714     Mar. 6, 1992
20715     May 21, 1997
20716    Feb. 18, 1998
20717    Jun. 18, 1998
20718    Mar. 30, 2001
Name: DATE OF BIRTH, Length: 20719, dtype: object


The above shows that they are clearly different, so we need to convert one to have the same format as the other. Lets modify salary_data to have year-month-day formatting.

In [32]:
# Convert salary_data["DATE OF BIRTH"] to year-month-day format

# Preprocess dates to ensure consistency: add a period after each month if it is missing
salary_data["DATE OF BIRTH"] = salary_data["DATE OF BIRTH"].apply(lambda x: x[:3] + '.' + x[3:] if x[3] != '.' else x)

# Convert date to datetime format
salary_data["DATE OF BIRTH"] = pd.to_datetime(salary_data["DATE OF BIRTH"], format='%b. %d, %Y')

# Convert date to year-month-day format
salary_data["DATE OF BIRTH"] = salary_data["DATE OF BIRTH"].dt.strftime('%Y-%m-%d')

salary_data["DATE OF BIRTH"]

0        1972-02-15
1        1980-05-02
2        1970-04-28
3        1977-03-18
4        1979-12-23
            ...    
20714    1992-03-06
20715    1997-05-21
20716    1998-02-18
20717    1998-06-18
20718    2001-03-30
Name: DATE OF BIRTH, Length: 20719, dtype: object

That looks good! After a bit preprocessing to make sure the formatting was consistent, we now have consistent birth date formats for each of our dataframes. It seems that we should be about ready to join.

## Merging X and Y's ##

We can now try to merge our Player_Metrics dataframe with our Salary_data df on columns [Player, Year, DOB]

In [33]:
# Convert all columns names from both dataframes to upper case
player_metrics.columns = player_metrics.columns.str.upper()
salary_data.columns = salary_data.columns.str.upper()

# Merge player_metrics and salary_data on columns Player, Year, Birth Date
merged_data = pd.merge(player_metrics, salary_data, how='inner', left_on=['PLAYER', 'YEAR', 'DATE OF BIRTH'], right_on=['PLAYER', 'SEASON', 'DATE OF BIRTH'], suffixes=('_x', '_y'))

In [35]:
# Compute the percentage of records we were able to join on
percentage_joined = merged_data.shape[0] / player_metrics.shape[0]
print(merged_data.shape[0], player_metrics.shape[0])
print(percentage_joined)

12152 15219
0.7984755897233721


Not bad! Looks like we were able to join on 12152 / 15219 possible records, which is about 79% of our player performance data.

## Saving this data back to disk ##

Our final step for the entity resolution process is going to be to save the merged data to disk.

In [36]:
# Save the merged data to a csv file
output_dir = '../../Data/Warehouse/MergedData'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_file = output_dir + '/merged_data.csv'
merged_data.to_csv(output_file, index=False)